In [1]:
import pandas as pd
import numpy as np

metadataDF = pd.read_csv('TheMoviesDataset/movies_metadata.csv', usecols=["id", "title", "budget", "genres", "popularity", "release_date", "revenue", "runtime", "vote_average", "vote_count"])

print("Headers: " + str(list(metadataDF.columns.values)))
print(metadataDF.head(2))

Headers: ['budget', 'genres', 'id', 'popularity', 'release_date', 'revenue', 'runtime', 'title', 'vote_average', 'vote_count']
     budget                                             genres    id  \
0  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   862   
1  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  8844   

  popularity release_date      revenue  runtime      title  vote_average  \
0  21.946943   1995-10-30  373554033.0     81.0  Toy Story           7.7   
1  17.015539   1995-12-15  262797249.0    104.0    Jumanji           6.9   

   vote_count  
0      5415.0  
1      2413.0  


C:\Users\danie\AppData\Local\Temp\ipykernel_10968\3047937051.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadataDF = pd.read_csv('TheMoviesDataset/movies_metadata.csv', usecols=["id", "title", "budget", "genres", "popularity", "release_date", "revenue", "runtime", "vote_average", "vote_count"])


In [2]:
#Count columns
import ast
from collections import Counter

#Copy DF for manipulation
df_copy = metadataDF.copy()

#Convert the list representations of genres into actual lists.
df_copy['genres'] = df_copy['genres'].apply(ast.literal_eval)
all_genres = [genre['name'] for sublist in df_copy['genres'] for genre in sublist]
genre_counts = Counter(all_genres)

genre_counts_df = pd.DataFrame(genre_counts.items(), columns=['Genre', 'Count'])
print(genre_counts_df.sort_values(by=['Count'], ascending=False))

                                    Genre  Count
6                                   Drama  20265
1                                  Comedy  13182
9                                Thriller   7624
5                                 Romance   6735
7                                  Action   6596
10                                 Horror   4673
8                                   Crime   4307
17                            Documentary   3932
3                               Adventure   3496
12                        Science Fiction   3049
2                                  Family   2770
13                                Mystery   2467
4                                 Fantasy   2313
0                               Animation   1935
15                                Foreign   1622
16                                  Music   1598
11                                History   1398
14                                    War   1323
18                                Western   1042
19                  

In [3]:
#Copy DF for manipulation
df_copy = metadataDF.copy()

#Convert the list representations of genres into actual lists.
df_copy['genres'] = df_copy['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

df_exploded_genres = df_copy.explode('genres')

#Extract genre name and drop original genre column
df_exploded_genres['genre_name'] = df_exploded_genres['genres'].apply(lambda x: x['name'] if isinstance(x,dict) else None)
df_exploded_genres = df_exploded_genres.drop(columns=['genres'])
#Drop rows where 'genre_name' is None
df_exploded_genres = df_exploded_genres.dropna(subset=['genre_name'])

#Ensure vote_average and popularity are numeric
df_exploded_genres['vote_average'] = pd.to_numeric(df_exploded_genres['vote_average'], errors='coerce')
df_exploded_genres['popularity'] = pd.to_numeric(df_exploded_genres['popularity'], errors='coerce')

#Drop rows where 'vote_average' or 'popularity' is NaN
df_exploded_genres = df_exploded_genres.dropna(subset=['vote_average', 'popularity'])

genre_stats = df_exploded_genres.groupby('genre_name').agg({
    'vote_average': 'mean',
    'popularity': 'mean'
}).reset_index()

print(genre_stats[['genre_name', 'vote_average']].sort_values(by=['vote_average'], ascending=False))
print("\n")
print(genre_stats[['genre_name', 'popularity']].sort_values(by=['popularity'], ascending=False))

         genre_name  vote_average
2         Animation      6.275181
10          History      6.154220
18              War      6.041119
6             Drama      5.905221
12            Music      5.879599
4             Crime      5.878407
5       Documentary      5.823093
13          Mystery      5.820470
14          Romance      5.802658
8           Fantasy      5.787722
7            Family      5.753394
9           Foreign      5.737978
3            Comedy      5.715741
1         Adventure      5.686070
17         Thriller      5.616713
0            Action      5.584167
15  Science Fiction      5.342895
16         TV Movie      5.236292
11           Horror      5.230394
19          Western      5.167179


         genre_name  popularity
1         Adventure    5.998335
8           Fantasy    5.363656
15  Science Fiction    4.999817
0            Action    4.771612
7            Family    4.729328
2         Animation    4.712107
17         Thriller    4.487624
4             Crime    4.144